In [1]:
from scrap import Scraper
from RAG_QnA import RAG_Model
scrp = Scraper()
rag = RAG_Model()

2024-08-05 14:32:08.239621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 14:32:08.263601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 14:32:08.271055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 14:32:10.481104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [19]:
url = "https://docs.portkey.ai/docs/guides/integrations/introduction-to-gpt-4o"

In [20]:
response = scrp.scrape_website(url=url)
if response==403:
    print("This Website don't allow scrapping")

In [21]:
response

200

In [22]:
rag.load_Database()

Its run..


In [26]:
rag.window_mem.clear()

In [28]:
print(rag.generateResponse("What is GPT-4o?"))

GPT-4o is an advanced AI model that handles a combination of text, audio, and video inputs and generates outputs in text, audio, and image formats. It supports text and image inputs currently, with additional modalities like audio being added soon. The model has improved function calling capabilities and can interpret and execute multiple commands in a more structured and efficient manner.</s>


In [25]:
print(rag.window_mem.buffer_as_str)

Human: What is langchain?
AI:  Langchain appears to be a platform that includes several components such as GPT-4o, Portkey, Supabase pgvector, and OpenAI SDK among others. It seems to be focused on building applications that utilize various models with different modalities like visual, audio, or text inputs. However, without more specific information about Langchain, it's hard to provide a more detailed explanation.</s>


In [1]:
from langchain.document_loaders import PyPDFLoader

In [10]:
import requests
def is_pdf_url(url):
    if url.lower().endswith('.pdf'):
        return True
    try:
        response = requests.head(url, allow_redirects=True)
        content_type = response.headers.get('Content-Type', '')
        return content_type.lower() == 'application/pdf'
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return False

In [12]:
is_pdf_url(url='https://www.cl.cam.ac.uk/teaching/1011/OpSystems/os1a-slides')

False

In [2]:
loader = PyPDFLoader(
    file_path='https://www.cl.cam.ac.uk/teaching/1011/OpSystems/os1a-slides.pdf'
)

In [3]:
loader.load()

[Document(metadata={'source': 'https://www.cl.cam.ac.uk/teaching/1011/OpSystems/os1a-slides.pdf', 'page': 0}, page_content='Operating Systems\nSteven Hand\nMichaelmas Term 2010\n12 lectures for CST IA\nOperating Systems — N/H/MWF@12'),
 Document(metadata={'source': 'https://www.cl.cam.ac.uk/teaching/1011/OpSystems/os1a-slides.pdf', 'page': 1}, page_content='Course Aims\n•This course aims to:\n–explain the structure and functions of an operating system,\n–illustrate key operating system aspects by concrete exampl e, and\n–prepare you for future courses. . .\n•At the end of the course you should be able to:\n–compare and contrast CPU scheduling algorithms\n–explain the following: process, address space, ﬁle.\n–distinguish paged and segmented virtual memory.\n–discuss the relative merits of Unix and NT. . .\nOperating Systems — Aims i'),
 Document(metadata={'source': 'https://www.cl.cam.ac.uk/teaching/1011/OpSystems/os1a-slides.pdf', 'page': 2}, page_content='Course Outline\n•Introduction

In [1]:
import warnings as wn
# Ignore warning messages
wn.filterwarnings('ignore')

from langchain_community.llms import HuggingFaceHub
from api_token import LargeLanguageModel
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain_openai import ChatOpenAI
import os
import openai

In [5]:
# Define filter terms to stop the generation
filter = [
        'Question:', 
        '**Question:**',
        'Human:']
        
        # Set up the language model endpoint
llm = HuggingFaceEndpoint(
        name="Web-Pilot",
        repo_id= 'mistralai/Mistral-Large-Instruct-2407',
        task="text-generation",
        huggingfacehub_api_token=LargeLanguageModel().get_Key(),
        verbose=False,
            # show output in text streaming
        streaming=True,
        temperature=0.9,
        return_full_text=True,
        max_new_tokens=500,
            # Stop sequences is filter for stop criteria
        stop_sequences=filter,
        repetition_penalty=1.1
    )

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [6]:
llm("Hi")

/home/junaid-ul-hassan/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


HfHubHTTPError:  (Request ID: 3eQpBk2YWFHALNlE6Ya6K)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/mistralai/Mistral-Large-Instruct-2407.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model mistralai/Mistral-Large-Instruct-2407 is too large to be loaded automatically (245GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [12]:
def remove_unwanted_suffixes(text):
    suffixes = ["</k>", "<|eot_id|>"]
    for suffix in suffixes:
        if text.endswith(suffix):
            return text[: -len(suffix)]
    return text

In [13]:
text = """(Remember to keep your answer concise and under 60 words)

Please type 'yes' if you'd like me to elaborate, otherwise, I'll proceed.

Type here:

type yes if you'd like me to elaborate.
else, i'll proceed.
waiting... <|eot_id|>"""

In [14]:
print(remove_unwanted_suffixes(text))

(Remember to keep your answer concise and under 60 words)

Please type 'yes' if you'd like me to elaborate, otherwise, I'll proceed.

Type here:

type yes if you'd like me to elaborate.
else, i'll proceed.
waiting... 
